In [2]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [119]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])


In [108]:
# Load the
video1 = cv2.VideoCapture('videos/1.mp4')
video2 = cv2.VideoCapture('videos/2.mp4')
video3 = cv2.VideoCapture('videos/3.mp4')
video4 = cv2.VideoCapture('videos/4.mp4')
video5 = cv2.VideoCapture('videos/5.mp4')

# Initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils


In [109]:
Y_labels = []
X_data = []
X_data_x = []
X_data_y = []

for n, video in enumerate([video1, video2, video3, video4, video5]):
    for i in range(int(video.get(cv2.CAP_PROP_FRAME_COUNT))):
        # Read each frame from the webcam
        _, frame = video.read()

        x, y, c = frame.shape

        # Flip the frame vertically
        frame = cv2.flip(frame, 1)
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get hand landmark prediction
        result = hands.process(framergb)


        # post process the result
        if result.multi_hand_landmarks:
            landmarks = []
            landmarks_x = np.zeros(21)
            landmarks_y = np.zeros(21)
            for handslms in result.multi_hand_landmarks:
                for i,lm in enumerate(handslms.landmark):
                    # print(id, lm)
                    lmx = int(lm.x * x)
                    lmy = int(lm.y * y)

                    landmarks.append([lmx, lmy])
                    landmarks_x[i]= int(lmx)                  
                    landmarks_y[i]= int(lmy)

            X_data.append((landmarks))
            X_data_x.append(landmarks_x)
            X_data_y.append(landmarks_y)
            Y_labels.append(n)

In [110]:
# X_data = np.array(X_data)
Y_labels = np.array(Y_labels)
# print(X_data.shape)
X_data_x = np.array(X_data_x)
X_data_y = np.array(X_data_y)
# print(X_data_x.shape)
# print(X_data_x)
X_data = np.concatenate((X_data_x, X_data_y), axis=1)
print(X_data.shape)
print(Y_labels.shape)

(1691, 42)
(1691,)


In [111]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [117]:
from sklearn.preprocessing import Normalizer


X_data1 = Normalizer().fit_transform(X_data)
X_data1, Y_labels = shuffle_in_unison(X_data1, Y_labels)
# Y_labels = tf.keras.utils.to_categorical(Y_labels, num_classes=5)
print(X_data1.shape)
print(Y_labels.shape)
print(Y_labels)

(1691, 42)
(1691,)
[4 1 3 ... 4 3 3]


In [120]:
model.predict(X_data1[0:1])

1/1 [==============================] - 0s 272ms/step


array([[-0.00178372,  0.05394217,  0.08625332,  0.07427401, -0.03657658]],
      dtype=float32)

In [121]:
# Train the model
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

model.fit(X_data, Y_labels, epochs=100)


Epoch 1/100


ValueError: in user code:

    File "c:\Users\thoma\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\thoma\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\thoma\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\thoma\anaconda3\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\thoma\anaconda3\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "c:\Users\thoma\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "

In [89]:
model.predict(X_data[0:1])

1/1 [==============================] - 0s 57ms/step


array([[-3.5463734 ,  0.31527212,  0.22629353,  0.3493707 ,  0.19577055,
         0.35709056]], dtype=float32)